# Generation of Haikus and of the matching images

This Jupyter Notebook is dedicated to the generation of the haikus with the trained models and the matching dataset and after that, to generate a corresponding image with a StableDiffusion model.


In [1]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
import re

import os

2024-09-03 19:55:29.901378: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-03 19:55:30.028361: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-03 19:55:30.062979: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-03 19:55:30.275526: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-03 19:55:31.975257: W tensorflow/compiler/tf2

## Loading the data for the tokeniser

In this first part, just like for the training notebook, we are loading the dataset corresponding to our model, so that the tokeniser is the same one as the one used for the training and to make the haikus make some sense.

In [2]:
# Load the CSV data
df = pd.read_csv('models/lstm_5000/haiku_dataset_1725314076.csv')

# Display the loaded data
df.head()

,line_1,line_2,line_3,source,line1_syllables,line2_syllables,line3_syllables,haiku
0,Says it all doesn't,it most of them don't even,know why they are there,twaiku,4,7,7,Says it all doesn't it most of them don't eve...
1,Cindy needed time,to take a nap on the floor,of the US Senate,twaiku,6,8,6,Cindy needed time to take a nap on the floor ...
2,A glorious morn,Without a cloud to be seen,Why then do I cry,twaiku,4,7,5,A glorious morn Without a cloud to be seen Wh...
3,Now playing We'll Meet,Again by Barry O'Dowd,The Shamrock Singers,twaiku,4,7,5,Now playing We'll Meet Again by Barry O'Dowd ...
4,Watching some courage,the cowardly dog show to,start the day off right,twaiku,7,7,5,Watching some courage the cowardly dog show t...


Now, we are cleaning our haikus after fusing them all, this will allow us to do the tokenization of our dataset after this. 

This is a way to do, but it is also possible to do it by using the three lines of the haikus and then having more differents lines and cases but for this first time, we will try to do it simple and then improve the project step by step with the time. 

In [3]:
# Combine all haikus into a single string
haikus = df['haiku'].tolist()
text = ' '.join(haikus)

# Clean the text
def clean_text(text):
    text = text.lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'\r', '', text)
    text = re.sub(r'\xa0', ' ', text)
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text)
    text = re.sub(r'\s+', ' ', text).strip()
    text = re.sub(r'[^\w\s]', '', text)
    return text

cleaned_text = clean_text(text)

As now our text are cleaned, we can tokenize them and see if the result is what we could expect from this process : 

In [4]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([cleaned_text])
total_words = len(tokenizer.word_index) + 1

# Convert haikus to sequences of tokens
input_sequences = []
for haiku in haikus:
    token_list = tokenizer.texts_to_sequences([clean_text(haiku)])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)

print(input_sequences[:10])

[[327, 16], [327, 16, 28], [327, 16, 28, 213], [327, 16, 28, 213, 16], [327, 16, 28, 213, 16, 186], [327, 16, 28, 213, 16, 186, 8], [327, 16, 28, 213, 16, 186, 8, 80], [327, 16, 28, 213, 16, 186, 8, 80, 29], [327, 16, 28, 213, 16, 186, 8, 80, 29, 76], [327, 16, 28, 213, 16, 186, 8, 80, 29, 76, 61]]


## Haikus generation

Now we will be generating a haikus using a random seed. 

This code is mainly here to show a single haiku generation but it is the same one as the training notebook.

In [6]:
# You can find the corresponding functions in the functions.py file
from functions import *

# Example usage
seed_word = select_random_seed_word_from_tokenizer(tokenizer)

params = {
    'model_path': 'models/lstm_5000/haiku_model_1725314076.keras',
    'seed_text': seed_word,
    'syllables_mode': True,
    'words_mode': False,
    'haikus_pattern': [5, 7, 5],
    'max_sequence_len': 21,
    'tokenizer': tokenizer  # Assuming tokenizer is already defined
}

seed_text, haiku = generate_haiku_from_params(params)
print(seed_text)
print(haiku)

meanwhile in the real world migration is back to pre crisis levels
['meanwhile in the', ' real world migration is back', ' to pre crisis levels']


## Generation of multiples haikus 

In this part we will be generating a given number of haikus in a folder that will be created to store them. 

We are doing so we will be able to identify each of them individually to after that generate the corresponding image with Stable Diffusion ! This is also to see what the haikus generated randomly can be about.

In [13]:
params = {
    'model_path': 'models/lstm_5000/haiku_model_1725314076.keras',
    'seed_text': seed_word,
    'syllables_mode': True,
    'words_mode': False,
    'haikus_pattern': [5, 7, 5],
    'max_sequence_len': 21,
    'tokenizer': tokenizer  # Assuming tokenizer is already defined
}

# Create a folder to save the haikus
output_folder = 'generated_haikus_100'
os.makedirs(output_folder, exist_ok=True)

# Generate and save 10 haikus
for i in range(100):
    seed_word = select_random_seed_word_from_tokenizer(tokenizer)
    params['seed_text'] = seed_word
    seed_text, haiku = generate_haiku_from_params(params)
    
    # Create a string with \n at the end of each element of the haiku list, stripping any leading/trailing whitespace
    haiku_string = "\n".join(line.strip() for line in haiku)

    # Save the haiku to a text file
    haiku_filename = os.path.join(output_folder, f'haiku_{i+1}.txt')
    with open(haiku_filename, 'w') as file:
        file.write(haiku_string)
    
    print(f'Haiku {i+1} saved to {haiku_filename}')

Haiku 1 saved to generated_haikus_100/haiku_1.txt
Haiku 2 saved to generated_haikus_100/haiku_2.txt
Haiku 3 saved to generated_haikus_100/haiku_3.txt
Haiku 4 saved to generated_haikus_100/haiku_4.txt
Haiku 5 saved to generated_haikus_100/haiku_5.txt
Haiku 6 saved to generated_haikus_100/haiku_6.txt
Haiku 7 saved to generated_haikus_100/haiku_7.txt
Haiku 8 saved to generated_haikus_100/haiku_8.txt
Haiku 9 saved to generated_haikus_100/haiku_9.txt
Haiku 10 saved to generated_haikus_100/haiku_10.txt
Haiku 11 saved to generated_haikus_100/haiku_11.txt
Haiku 12 saved to generated_haikus_100/haiku_12.txt
Haiku 13 saved to generated_haikus_100/haiku_13.txt
Haiku 14 saved to generated_haikus_100/haiku_14.txt
Haiku 15 saved to generated_haikus_100/haiku_15.txt
Haiku 16 saved to generated_haikus_100/haiku_16.txt
Haiku 17 saved to generated_haikus_100/haiku_17.txt
Haiku 18 saved to generated_haikus_100/haiku_18.txt
Haiku 19 saved to generated_haikus_100/haiku_19.txt
Haiku 20 saved to generated_ha

## Image from haikus generation

In this final part, we will be loading the pipeline of Stable Diffusion and then open and read all of the haikus to then generate a corresponding image with the pipeline.

In [11]:
import torch
from diffusers import StableDiffusionPipeline, DDPMScheduler
from PIL import Image

pipe = StableDiffusionPipeline.from_pretrained("CompVis/stable-diffusion-v1-4", variant="fp16", torch_dtype=torch.float16).to("cuda")

print("Stable Diffusion pipeline loaded")

/home/hook/.pyenv/versions/3.10.13/envs/haikus/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]/home/hook/.pyenv/versions/3.10.13/envs/haikus/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Loading pipeline components...: 100%|██████████| 7/7 [00:01<00:00,  3.80it/s]


Stable Diffusion pipeline loaded


In [14]:
# Folder containing the haikus
haiku_folder = 'generated_haikus_100'

# List all .txt files in the folder
haiku_files = [f for f in os.listdir(haiku_folder) if f.endswith('.txt')]

# Function to generate image from text
def generate_image_from_text(text, output_path):
    prompt = text
    scheduler = DDPMScheduler(beta_start=0.00085, beta_end=0.012, beta_schedule="scaled_linear")
    image = pipe(prompt, scheduler=scheduler, num_inference_steps=50, guidance_scale=5.0).images[0]
    image.save(output_path)

# Iterate over each haiku file and generate the corresponding image
for haiku_file in haiku_files:
    haiku_path = os.path.join(haiku_folder, haiku_file)
    
    # Read the haiku text
    with open(haiku_path, 'r') as file:
        haiku_text = file.read()
    
    # Generate the image
    image_output_path = os.path.join(haiku_folder, haiku_file.replace('.txt', '.png'))
    generate_image_from_text(haiku_text, image_output_path)
    
    print(f'Image for {haiku_file} saved to {image_output_path}')

100%|██████████| 50/50 [07:51<00:00,  9.42s/it]


Image for haiku_10.txt saved to generated_haikus_100/haiku_10.png


100%|██████████| 50/50 [07:24<00:00,  8.89s/it]


Image for haiku_12.txt saved to generated_haikus_100/haiku_12.png


100%|██████████| 50/50 [07:08<00:00,  8.56s/it]


...

Image for haiku_43.txt saved to generated_haikus_100/haiku_43.png


100%|██████████| 50/50 [06:25<00:00,  7.71s/it]


Image for haiku_20.txt saved to generated_haikus_100/haiku_20.png


100%|██████████| 50/50 [07:14<00:00,  8.68s/it]


Image for haiku_28.txt saved to generated_haikus_100/haiku_28.png


This really simple last part is mostly just to play with Stable Diffusion and see some funny results that can be generated with it and the generated haikus. The results were fun and this it was a really nice thing to discover as I never used Stable Diffusion models before !